In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import eda packages

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from functools import partial

# import deep learning packages

from keras.layers import Dense, Dropout, Input, MaxPool2D, GlobalAveragePooling2D, ZeroPadding2D, Conv2D, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB6, VGG16, ResNet101, ResNet50, EfficientNetB0
from keras.optimizers import Adam
!pip install tensorflow_addons
from tensorflow_addons.optimizers import Yogi
from keras import Model, regularizers
from tensorflow.keras import layers

In [ ]:
#navigate to the correct folder for training data
%cd /kaggle/input/cassava-leaf-disease-classification/train_tfrecords

In [ ]:
import tensorflow as tf
from functools import partial
import matplotlib.pyplot as plt
BATCH_SIZE = 32
IMAGE_SIZE = (512, 512)
TRAIN_FILENAMES = ['ld_train13-1338.tfrec',
 'ld_train15-1327.tfrec',
 'ld_train07-1338.tfrec',
 'ld_train04-1338.tfrec',
 'ld_train05-1338.tfrec',
 'ld_train00-1338.tfrec',
 'ld_train08-1338.tfrec',
 'ld_train06-1338.tfrec',
 'ld_train03-1338.tfrec',
 'ld_train09-1338.tfrec',
 'ld_train02-1338.tfrec',
 'ld_train11-1338.tfrec',
 'ld_train01-1338.tfrec',
 'ld_train12-1338.tfrec']
VALIDATION_FILENAMES = ['ld_train14-1338.tfrec', 'ld_train10-1338.tfrec'] 

In [ ]:
#Code from https://keras.io/examples/keras_recipes/tfrecord/

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    return image

def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image": tf.io.FixedLenFeature([], tf.string),
            "target": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        label = tf.cast(example["target"], tf.int32)
        return image, label
    return image

def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=1
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset


def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(100)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

In [ ]:
#create the train and validation sets
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = get_dataset(TRAIN_FILENAMES)
validation_dataset = get_dataset(VALIDATION_FILENAMES)

In [ ]:
import keras, tensorflow
from keras.models import load_model

models = list()
%cd ~

model1 = tensorflow.keras.models.load_model("/kaggle/input/model-eff/model_unfrozen_tf")
model2 = tensorflow.keras.models.load_model("/kaggle/input/model-res/model2_unfrozen_tf")
model3 = tensorflow.keras.models.load_model("/kaggle/input/model-vgg/model3_unfrozen_tf")

In [ ]:
%cd /kaggle/input/cassava-leaf-disease-classification/test_tfrecords/

TEST_FILENAMES = ['ld_test00-1.tfrec'] 
test_dataset = load_dataset(TEST_FILENAMES, labeled = False)
test_dataset = test_dataset.shuffle(100)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

models = list()

models.append(model1)
models.append(model2)
models.append(model3)

preds = []

sample_sub = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')

for image in sample_sub.image_id:
    img = keras.preprocessing.image.load_img('/kaggle/input/cassava-leaf-disease-classification/test_images/' + image, target_size = (224, 224))
    yhats = [model.predict(test_dataset) for model in models]
    yhats = np.array(yhats)
    summed = np.sum(yhats, axis=0)
    preds.append(np.argmax(summed, axis=1))

my_submission = pd.DataFrame({'image_id': sample_sub.image_id, 'label': preds})
my_submission.to_csv('/kaggle/working/submission.csv', index=False)